<a href="https://colab.research.google.com/github/hyulianton/JaringanSyarafTiruan/blob/main/jst_Implementasi_Multilayer_Perceptron_(MLP)_dari_Nol_dengan_NumPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ini adalah proyek implementasi Jaringan Saraf Tiruan dua lapisan (satu _hidden layer_) yang sepenuhnya menggunakan **NumPy**. Tujuannya adalah memahami bagaimana operasi matriks, _forward pass_, dan terutama _backward pass_ (Backpropagation) bekerja tanpa bantuan _framework_ otomatis seperti PyTorch atau TensorFlow.

## **Bagian 0: Setup dan Mendapatkan Data**

Langkah pertama, kita akan menyiapkan lingkungan dan memuat _dataset_ Iris yang terkenal untuk tugas klasifikasi tiga kelas.

### **Sel 0.1: Impor Library**

Kita hanya memerlukan `numpy` untuk komputasi dan `sklearn` untuk memuat data dan _preprocessing_.


In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

### **Sel 0.2: Mendownload dan Memuat Dataset Iris**

_Dataset_ Iris memiliki 4 fitur (panjang/lebar sepal dan petal) dan 3 kelas bunga (Setosa, Versicolor, Virginica).

In [2]:
# 1. Muat Dataset
iris = load_iris()
X = iris.data   # Fitur (4 kolom)
y = iris.target # Label (0, 1, 2)

# 2. Preprocessing Data
# Normalisasi Fitur: Penting agar Gradien Descent bekerja dengan baik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encoding Label (One-Hot Encoding): Mengubah label 0, 1, 2 menjadi vektor [1,0,0], [0,1,0], [0,0,1]
# Ini diperlukan untuk klasifikasi multi-kelas dengan Cross-Entropy Loss
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# 3. Membagi Data: Pelatihan (80%) dan Pengujian (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

# Cek Dimensi Data
print(f"Dimensi Data Pelatihan (X_train): {X_train.shape}") # (120, 4)
print(f"Dimensi Label Pelatihan (y_train): {y_train.shape}") # (120, 3)

Dimensi Data Pelatihan (X_train): (120, 4)
Dimensi Label Pelatihan (y_train): (120, 3)


**Penjelasan:** _Dataset_ diproses agar siap untuk MLP: fitur distandarisasi dan label di-_encode_ menjadi format vektor (One-Hot) yang sesuai untuk _output layer_ klasifikasi.

----------

## **Bagian 1: Definisi Fungsi Inti**

Kita harus membuat fungsi-fungsi matematis yang diperlukan oleh MLP: fungsi aktivasi, dan turunannya (untuk _backward pass_).

### **Sel 1.1: Fungsi Aktivasi**

Kita akan menggunakan **ReLU** untuk _hidden layer_ dan **Softmax** untuk _output layer_.

In [3]:
def relu(Z):
    """Fungsi Aktivasi Rectified Linear Unit (ReLU)"""
    return np.maximum(0, Z)

def relu_derivative(Z):
    """Turunan ReLU (untuk Backpropagation)"""
    return (Z > 0).astype(float)

def softmax(Z):
    """Fungsi Aktivasi Softmax (untuk Output Klasifikasi Multi-kelas)"""
    # Kurangi nilai Z dengan nilai maksimum untuk stabilitas numerik
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

**Penjelasan:** Turunan `relu` sangat sederhana: 1 jika input positif, dan 0 jika negatif atau nol. `softmax` mengubah skor output menjadi probabilitas yang totalnya 1.

### **Sel 1.2: Fungsi Rugi (Loss Function)**

Untuk klasifikasi multi-kelas (Iris), kita menggunakan **Categorical Cross-Entropy Loss**.

In [4]:
def cross_entropy_loss(Y_true, Y_pred):
    """Categorical Cross-Entropy Loss"""
    # Jumlahkan kerugian dari setiap sampel
    m = Y_true.shape[0]
    # Hindari log(0)
    Y_pred = np.clip(Y_pred, 1e-12, 1. - 1e-12)

    # Rumus Cross-Entropy Loss
    loss = -np.sum(Y_true * np.log(Y_pred)) / m
    return loss

## **Bagian 2: Inisialisasi dan Arsitektur MLP**

Kita mendefinisikan arsitektur 4-H-3 (4 input, H _hidden units_, 3 output) dan menginisialisasi parameter.

### **Sel 2.1: Inisialisasi Parameter (Bobot & Bias)**

Kita menggunakan inisialisasi acak, yang sangat penting untuk memecah simetri.

In [6]:
# Arsitektur Jaringan
input_size = X_train.shape[1]  # 4 fitur
hidden_size = 10               # 10 neuron di Hidden Layer
output_size = y_train.shape[1] # 3 kelas

def initialize_parameters(input_size, hidden_size, output_size):
    # Inisialisasi He/Kaiming (cocok untuk ReLU)
    W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
    b2 = np.zeros((1, output_size))

    parameters = {
        "W1": W1, "b1": b1,
        "W2": W2, "b2": b2
    }
    return parameters

parameters = initialize_parameters(input_size, hidden_size, output_size)
print("Parameter W1 shape:", parameters["W1"].shape)
print("Parameter W2 shape:", parameters["W2"].shape)

Parameter W1 shape: (4, 10)
Parameter W2 shape: (10, 3)


## **Bagian 3: Forward dan Backward Pass**

Ini adalah jantung dari seluruh model. Kita mendefinisikan bagaimana data mengalir (Forward) dan bagaimana gradien dihitung (Backward).

### **Sel 3.1: Forward Pass (Maju ke Depan)**

In [7]:
def forward_pass(X, parameters):
    W1, b1, W2, b2 = parameters["W1"], parameters["b1"], parameters["W2"], parameters["b2"]

    # Lapisan 1 (Hidden Layer)
    Z1 = X @ W1 + b1  # X @ W1 adalah Perkalian Matriks (vektorisasi)
    A1 = relu(Z1)     # Aktivasi ReLU

    # Lapisan 2 (Output Layer)
    Z2 = A1 @ W2 + b2
    A2 = softmax(Z2)  # Aktivasi Softmax menghasilkan Y_pred

    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2, "X": X}
    return A2, cache # A2 adalah Y_pred


### **Sel 3.2: Backward Pass (Backpropagation)**

Ini adalah bagian terumit. Kita menggunakan **Chain Rule** untuk menghitung gradien.

In [8]:
def backward_pass(Y_pred, Y_true, parameters, cache):
    m = Y_pred.shape[0]  # Jumlah sampel (Batch Size)

    W1, W2 = parameters["W1"], parameters["W2"]
    A1, X = cache["A1"], cache["X"]
    Z1 = cache["Z1"]

    grads = {}

    # 1. Gradien Lapisan Output (W2, b2)
    # Gradien Loss terhadap Z2 (Sinyal Delta Delta(2))
    # Untuk Cross-Entropy + Softmax, turunannya SANGAT sederhana:
    dZ2 = Y_pred - Y_true # (Y_pred - Y_true)

    # Gradien Bobot W2
    grads["dW2"] = (A1.T @ dZ2) / m
    # Gradien Bias b2
    grads["db2"] = np.sum(dZ2, axis=0, keepdims=True) / m

    # 2. Gradien Lapisan Tersembunyi (W1, b1)
    # Sebarkan Sinyal Delta ke Lapisan 1
    dA1 = dZ2 @ W2.T

    # Turunan ReLU * Sinyal Delta = Sinyal Delta(1)
    dZ1 = dA1 * relu_derivative(Z1)

    # Gradien Bobot W1
    grads["dW1"] = (X.T @ dZ1) / m
    # Gradien Bias b1
    grads["db1"] = np.sum(dZ1, axis=0, keepdims=True) / m

    return grads


**Penjelasan Backpropagation:**

1.  **Output:** Turunan _Loss_ dihitung pertama. Untuk Cross-Entropy dengan Softmax, ∂Z2​∂L​ hanya sebatas `Y_pred - Y_true`.
    
2.  **Sebar Delta:** Sinyal kesalahan `dZ2` disebarkan ke lapisan sebelumnya menggunakan perkalian matriks dengan Bobot W2​ yang ditranspos → `dA1`.
    
3.  **ReLU Lokal:** `dA1` dikalikan dengan turunan fungsi aktivasi ReLU lokal → `dZ1` (Sinyal Delta Lapisan 1).
    
4.  **W1/b1:** Gradien W1 dan b1 dihitung menggunakan `dZ1` dan `X`.
    

----------

## **Bagian 4: Pelatihan (Training Loop)**

Kita menyatukan semua fungsi ke dalam Siklus Pelatihan (_Training Loop_) utama.

### **Sel 4.1: Gradient Descent dan Training**


In [9]:
def train_mlp(X_train, y_train, parameters, learning_rate, epochs):
    history = []

    for epoch in range(epochs):
        # 1. FORWARD PASS
        Y_pred, cache = forward_pass(X_train, parameters)

        # 2. HITUNG LOSS
        loss = cross_entropy_loss(y_train, Y_pred)

        # 3. BACKWARD PASS
        grads = backward_pass(Y_pred, y_train, parameters, cache)

        # 4. UPDATE PARAMETER (Gradient Descent)
        # W_baru = W_lama - eta * Gradien
        parameters["W1"] -= learning_rate * grads["dW1"]
        parameters["b1"] -= learning_rate * grads["db1"]
        parameters["W2"] -= learning_rate * grads["dW2"]
        parameters["b2"] -= learning_rate * grads["db2"]

        history.append(loss)

        if epoch % 1000 == 0:
            print(f"Epoch {epoch}/{epochs} | Loss: {loss:.4f}")

    return parameters, history

# Hyperparameters
learning_rate = 0.1
epochs = 10000

# Jalankan Pelatihan
trained_parameters, loss_history = train_mlp(X_train, y_train, parameters, learning_rate, epochs)


Epoch 0/10000 | Loss: 1.3943
Epoch 1000/10000 | Loss: 0.0537
Epoch 2000/10000 | Loss: 0.0464
Epoch 3000/10000 | Loss: 0.0439
Epoch 4000/10000 | Loss: 0.0427
Epoch 5000/10000 | Loss: 0.0419
Epoch 6000/10000 | Loss: 0.0414
Epoch 7000/10000 | Loss: 0.0411
Epoch 8000/10000 | Loss: 0.0408
Epoch 9000/10000 | Loss: 0.0406


### **Sel 4.2: Evaluasi Model**

Setelah pelatihan, kita hitung akurasi pada data uji.


In [10]:
def evaluate(X, y_true, parameters):
    # Lakukan Forward Pass pada data
    Y_pred_prob, _ = forward_pass(X, parameters)

    # Konversi probabilitas (Y_pred_prob) ke label kelas (indeks dengan probabilitas tertinggi)
    y_pred_labels = np.argmax(Y_pred_prob, axis=1)
    y_true_labels = np.argmax(y_true, axis=1) # y_true juga perlu dikonversi dari One-Hot

    # Hitung Akurasi
    accuracy = np.mean(y_pred_labels == y_true_labels)
    return accuracy

# Evaluasi pada Data Uji
accuracy = evaluate(X_test, y_test, trained_parameters)
print(f"\n--- HASIL AKHIR ---")
print(f"Akurasi Model pada Data Uji: {accuracy*100:.2f}%")



--- HASIL AKHIR ---
Akurasi Model pada Data Uji: 96.67%


## **Kesimpulan dan Tindak Lanjut**

Anda telah berhasil mengimplementasikan seluruh siklus pelatihan MLP 2-lapisan menggunakan NumPy. Hasil akurasi yang tinggi (sekitar 90-100% untuk Iris) menunjukkan bahwa **Backpropagation Anda bekerja dengan benar**.

**Tantangan Selanjutnya:**

1.  **Plot Loss:** Visualisasikan `loss_history` untuk melihat konvergensi.
    
2.  **Gradient Checking:** Tambahkan implementasi _Gradient Checking_ untuk memverifikasi `backward_pass` Anda.
    
3.  **Regulasi:** Tambahkan L2 Regularization pada _Loss_ dan _Backward Pass_ untuk mengontrol _overfitting_.
